In [23]:
library(tidyverse)
library(palmerpenguins)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)


── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks readr::spec()
✖ recipes::step()   masks stats::step()
• Use suppressPackageStartupMessages() to eliminate package startup messages



In [10]:
penguins

species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
<fct>,<fct>,<dbl>,<dbl>,<int>,<int>,<fct>,<int>
Adelie,Torgersen,39.1,18.7,181,3750,male,2007
Adelie,Torgersen,39.5,17.4,186,3800,female,2007
Adelie,Torgersen,40.3,18.0,195,3250,female,2007
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
Chinstrap,Dream,49.6,18.2,193,3775,male,2009
Chinstrap,Dream,50.8,19.0,210,4100,male,2009
Chinstrap,Dream,50.2,18.7,198,3775,female,2009


In [39]:
penguins_recoded <- penguins |>
    mutate(year = as_factor(year)) |>
    mutate(year = fct_recode(year, "7" = "2007", "8" = "2008", "9" = "2009"))
penguins_recoded
           


Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `year = fct_recode(year, `7` = "2007", `8` = "2008", `9` =
  "2009")`.
Caused by warning:
! Unknown levels in `f`: 2007, 2008, 2009”


species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
<fct>,<fct>,<dbl>,<dbl>,<int>,<int>,<fct>,<fct>
Adelie,Torgersen,39.1,18.7,181,3750,male,7
Adelie,Torgersen,39.5,17.4,186,3800,female,7
Adelie,Torgersen,40.3,18.0,195,3250,female,7
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
Chinstrap,Dream,49.6,18.2,193,3775,male,9
Chinstrap,Dream,50.8,19.0,210,4100,male,9
Chinstrap,Dream,50.2,18.7,198,3775,female,9


In [64]:
library(themis)
new_penguins <- tibble(bill_length_mm = c(38, 50, 29, 45), bill_depth_mm = c(19, 18, 14, 14))

rare_penguins <- bind_rows(
      filter(penguins, species == "Gentoo" | species == "Chinstrap"),
      penguins |> filter(species == "Adelie") |> slice_head(n = 3)
    ) |>
    select(species, bill_length_mm, bill_depth_mm)

penguins_recipe <- recipe(species ~ bill_length_mm + bill_depth_mm, data = rare_penguins) |>
        step_upsample(species, over_ratio = 0.1, skip = FALSE) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors()) 

rare_penguins_prepped <- penguins_recipe |>
    prep() |>
    bake(rare_penguins)

numbers <- rare_penguins_prepped |>
    group_by(species) |>
    summarize(count = n())
numbers 

rare_penguins_prepped
# knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 5) |>
#     set_engine("kknn") |>
#     set_mode("classification")

# penguins_scaled <- penguins_recipe |>
#     prep() |>
#     bake(penguins)
# penguins_scaled

# penguins_workflow <- workflow() |>
#     add_recipe(penguins_recipe) |>
#     add_model(knn_spec) |>
#     fit(rare_penguins)

# predict(penguins_workflow, new_penguins)


# penguin_plot <- rare_penguins |>
#     ggplot(aes(x = bill_length_mm, y = bill_depth_mm)) +
#     geom_point(aes(color = species)) +
#     labs(color = "Penguin Species", x = "Bill Length (Scaled)", y = "Bill Depth (Scaled)") +
#     ggtitle("Penguin species tend to have distinctive bill lengths and depths") +

#     theme(text = element_text(size = 12))
# penguin_plot

species,count
<fct>,<int>
Adelie,12
Chinstrap,68
Gentoo,124


bill_length_mm,bill_depth_mm,species
<dbl>,<dbl>,<fct>
-2.053405,0.8482277,Adelie
-2.282785,0.5285677,Adelie
-2.053405,0.8482277,Adelie
⋮,⋮,⋮
0.6131402,-0.2173059,Gentoo
0.3550874,-1.2295628,Gentoo
0.3550874,-1.2295628,Gentoo
